In [4]:
# !pip install -r https://raw.githubusercontent.com/anujavenkatachalam04/chvi_vbd_rj/main/requirements.txt

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='pypi.esri.com', port=443): Max retries exceeded with url: /simple/arcgis/ (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x7cf712ced940>: Failed to establish a new connection: [Errno -2] Name or service not known'))



In [2]:
!pip install retry_requests

In [3]:
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')
import geopandas as gpd
from shapely.geometry import Point
import uuid
from dotenv import load_dotenv
import openmeteo_requests
import requests_cache
from retry_requests import retry
from tqdm import tqdm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.chdir("/content/drive/MyDrive/CHVI")


In [5]:
blocks_shp=gpd.read_file("5_Shapefiles/SUBDISTRICT_11/Rajasthan_Blocks.geojson")

In [6]:
grid=gpd.read_file("5_Shapefiles/Rajasthan_1deg.geojson")

In [ ]:
# Load API key from .env  only works for gh
# load_dotenv()
# OPENMETEO_API_KEY = os.getenv("OPENMETEO_API_KEY")

In [7]:
grid

,NAME,DISTRICT,C_CODE11,grid_id,geometry
0,Jaisalmer,Jaisalmer,0811400574000000,111,POINT (69.58154 26.76008)
1,Jaisalmer,Jaisalmer,0811400574000000,112,POINT (69.58154 26.86008)
2,Jaisalmer,Jaisalmer,0811400574000000,113,POINT (69.58154 26.96008)
3,Jaisalmer,Jaisalmer,0811400574000000,114,POINT (69.58154 27.06008)
4,Jaisalmer,Jaisalmer,0811400574000000,115,POINT (69.58154 27.16008)
...,...,...,...,...,...
3090,Dhaulpur,Dhaulpur,0810600518000000,6243,POINT (77.98154 26.76008)
3091,Dhaulpur,Dhaulpur,0810600518000000,6244,POINT (77.98154 26.86008)
3092,Rajakhera,Dhaulpur,0810600519000000,6316,POINT (78.08154 26.76008)
3093,Rajakhera,Dhaulpur,0810600519000000,6317,POINT (78.08154 26.86008)


# Extract Daily ERA5-Land data at 0.1 degree resolution

In [18]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup Open-Meteo API client
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


def fetch_daily_era5_data(lat, lon, start_date, end_date, name, grid_id, district, code, api_key, model, daily_vars):
    """
    Fetch daily ERA5 data for a single coordinate (lat/lon)
    Returns a Pandas DataFrame with all variables and metadata.
    """

    url = "https://customer-archive-api.open-meteo.com/v1/archive"

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "daily": daily_vars,
        "models": model,
        "apikey": api_key
    }

    try:
        # Call API
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]
        daily = response.Daily()

        # Build time range
        date_index = pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        )

        # Map variables dynamically based on list order
        daily_data = {"date": date_index}
        for i, var in enumerate(daily_vars):
            daily_data[var] = daily.Variables(i).ValuesAsNumpy()

        # Add metadata
        daily_data.update({
            "NAME": name,
            "DISTRICT": district,
            "C_CODE11": code,
            "grid_id": grid_id,
            "latitude": lat,
            "longitude": lon
        })

        return pd.DataFrame(daily_data)

    except Exception as e:
        print(f"Error fetching data for {name} ({district}): {e}")
        return pd.DataFrame()


In [1]:
start_date = "2020-01-01"
end_date = "2025-11-17"
api_key=""


all_results = []
for idx, row in tqdm(grid.iterrows(), total=len(grid)):
    lat = row.geometry.y
    lon = row.geometry.x
    sdtname = row.NAME
    dtname = row.DISTRICT
    sdtcode = row.C_CODE11
    grid_id=row.grid_id

    df = fetch_daily_era5_data(lat, lon, start_date, end_date, sdtname, grid_id, dtname, sdtcode, api_key, "era5_land", ["temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "precipitation_sum", "relative_humidity_2m_mean", "wind_speed_10m_mean", "wind_speed_10m_max"])
    if not df.empty:
        all_results.append(df)

# Combine
era5_grid_df = pd.concat(all_results, ignore_index=True)

In [18]:
era5_grid_df.date.max()

Timestamp('2025-11-17 00:00:00+0000', tz='UTC')

In [ ]:
era5_grid_df["diurnal_temperature_range_computed"]=era5_grid_df["temperature_2m_max"]-era5_grid_df["temperature_2m_min"]

In [19]:
era5_grid_df.to_csv("1_Data/Climate/Raw/openmeteo_daily_era5land_1deg_grid_2020-01-01_to_2025-11-17.csv", index=False)

# Extract Hourly ERA5-Land data at 0.1 degree resolution

In [ ]:
# The End!